In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from math import sin
from math import radians
from matplotlib import pyplot
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.arima.model import ARIMA 
from sklearn.metrics import mean_absolute_error
import pandas_datareader.data as pdr

#### Timeseries Analysis using Statistics.

**Preprocessing Steps for Timeseries Analysis.**
In time series, we need to make the timeseries stationary. For that we need to remove trends and seasonality in timeseries data. 

    Trends: 
        Linear Trend
        Exponential Trend (Apply log function to convert the exponential to linear)
        
    Seasional:
        Periods
        
***Remove Trend/Seasonality in timeseries***

    Methods:
        1. Subtract the rolling mean
        2. Differencing
        3. Decomposition.
        
You can use any of the above methods to remove trend and make it stationary. 
     
     
**Stat Models**

    1. AR Model
    2. MA Model
    3. ARMA Model
    4. ARIMA Model
    5. ARCH Model
    6. GRACH Model

#### Load Data

In [ ]:
def dateparser(s):
    return datetime.strptime(s, '%d.%m.%Y %H:%M:%S')

In [ ]:
data = pd.read_csv('../datasets/AirPassengers.csv', parse_dates=['Month'], index_col='Month')
data.head(2)

#### Check Stationarity.

Verify constant mean, constant variance and autocovariance not a function of time t. 

Ways to validate:
    1. Visual Representaion
    2. Dickey-Fuller Test(Statistical Test)

The Augmented Dickey-Fuller test can be used to test for a unit root in a univariate process in the presence of serial correlation.

**Visual Representaion**

In [ ]:
data.info()

In [ ]:
ts = data['#Passengers'] 
plt.plot(ts)
plt.show()

As you can see, mean and variance of this time series is changing over time and it is a linear trend  

**Dickey-Fuller Test**

In [ ]:
#Perform Dickey-Fuller test:
dftest = adfuller(ts, autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])

for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

# Here null hypothesis is there exists a unit root in time series data. 
# If p-value > 0.5, for sure there exists a unit root.
# Check if p-value < alpha (1%, 5% 10%) to reject the null hypothesis. 

Here p-value is greater than 0.5, which means there exists a unit root. This timeseries is not stationary.

#### Remove Trends

In [ ]:
# Apply log to make it linear trend.
ts_log = np.log(ts)
plt.plot(ts_log)

#### Normalization

${n}_t = \frac{{v}_t - \mu}{\sigma}$


In [ ]:
mean = np.mean(ts_log)
std = np.std(ts_log)

ts_norm = (ts_log - mean)/std
plt.plot(ts_norm)

In [ ]:
#Perform Dickey-Fuller test:
dftest = adfuller(ts_norm, autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])

for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

**1. Subtract the rolling mean**

In [ ]:
# moving average
moving_avg = ts_norm.rolling(window=12, center=False).mean()
plt.plot(ts_norm)
plt.plot(moving_avg, color='red')

#rolstd = ts_log.rolling(window=12, center=False).std()
#plt.plot(rolstd, color='black', label= Rolling Std')
#plt.legend(loc='best')

plt.show()

In [ ]:
# Remove trend from original data
ts_log_moving_avg_diff = ts_norm - moving_avg
ts_log_moving_avg_diff.dropna(inplace=True)

# Calculate current mean for plotting. 
moving_avg = ts_log_moving_avg_diff.rolling(window=12, center=False).mean()
rolstd = ts_log_moving_avg_diff.rolling(window=12, center=False).std()

# Plot
plt.plot(ts_log_moving_avg_diff, label="diff orig")
plt.plot(moving_avg, color='red', label='mean')
plt.legend(loc='best')
plt.show()

**2. Differencing**

Differencing can help stabilize the mean of the time series by removing changes in the level of a time series, and so eliminating (or reducing) trend and seasonality

Lag Difference

    Taking the difference between consecutive observations is called a lag-1 difference.

    The lag difference can be adjusted to suit the specific temporal structure.

    For time series with a seasonal component, the lag may be expected to be the period (width) of the seasonality.

Difference Order

    Some temporal structure may still exist after performing a differencing operation, such as in the case 
    of a nonlinear trend.

    As such, the process of differencing can be repeated more than once until all temporal dependence has 
    been removed.

    The number of times that differencing is performed is called the difference order.

In [ ]:
# create a differenced series
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return pd.Series(diff)

# invert differenced forecast
def inverse_difference(last_ob, value):
    return value + last_ob

In [ ]:
# Differencing with custom function.
diffdata = difference(ts_norm, interval = 1)
pyplot.plot(diffdata)
pyplot.show()

In [ ]:
# Another way to do differencing.
diff = ts_norm.diff(periods=1).dropna()
moving_avg = diff.rolling(window=12, center=False).mean()

pyplot.plot(diff)
plt.plot(moving_avg, color='red', label='mean')
pyplot.show()

In [ ]:
#Perform Dickey-Fuller test:
dftest = adfuller(diff, autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])

for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

**3. Decomposition**

In [ ]:
decomposition = seasonal_decompose(ts_norm, model='additive', period=12)
trend = decomposition.trend.dropna()
seasonal = decomposition.seasonal.dropna()
residual = decomposition.resid.dropna()
plt.figure(figsize=(10,6))

plt.subplot(411)
plt.plot(ts_log, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()


In [ ]:
print(trend)
print(seasonal)
print(residual)

In [ ]:
moving_avg = residual.rolling(window=12, center=False).mean()
pyplot.plot(residual)
plt.plot(moving_avg, color='red', label='mean')

In [ ]:
dftest = adfuller(residual, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

#### Fit Models

In [ ]:
# Prepare Train and Test Data
# In timeseries data, make sure when we split the data, order is not changed.
# We need to maintain order of the timeseries.
train_end_date = datetime(1959, 12, 1)
test_date = datetime(1960, 12, 1)

train_data = residual[:train_end_date]
test_data = ts[train_end_date + timedelta(days=1):test_date]

train_data.index = pd.DatetimeIndex(train_data.index.values, freq=train_data.index.inferred_freq)

print(train_data)
print(test_data)

**1. AR Model**

In [ ]:
# Find the order for AR model using PACF. 
plot_pacf(train_data)
plt.show()

By looking at this plot, we can set order to 8.

In [ ]:
armodel = ARIMA(train_data, order=(4,0,0))
armodel_fit = armodel.fit()
print(armodel_fit.summary())

#### Inference

Operations applied on timeseries data.

*1. Exponential trend to linear by applying log function.*
    
${l}_t = \log({{x}_t})$
    
*2. Normalize the data*

${n}_t = \frac{{v}_t - \mu}{\sigma}$

*3. Seasonal Decomposition*
    
    trend
    seasional
    residual


Now, after forecasting the data, convert the results back to its original form.

1. Results = ${y}_t$

2. Add trend, seasionality to results.

3. ${o}_t = {y}_t .{\sigma} + \mu$

4. $\varepsilon^{{o}_t}$

In [ ]:
# Forecast for next 12 months
results = armodel_fit.forecast(6)
results

In [ ]:
# Undo transformations. In decomposition, 
# as it does symmetry moving average, it will remove first 6 months and last 6 months of data.
trend1 = trend[train_end_date + timedelta(days=1):test_date]
seasonal1 = seasonal[train_end_date + timedelta(days=1):test_date]
results = results + trend1 + seasonal1
predictions = np.exp(results * std + mean)
predictions = predictions.dropna()

In [ ]:
mean_absolute_error(predictions, test_data[:6])

In [ ]:
plt.figure(figsize=(10,4))

plt.plot(ts)
plt.plot(predictions)

plt.legend(('Data', 'Predictions'), fontsize=16)

#### Stock Price Predictions..

#### Read Stock Data

In [ ]:
start = datetime(2015, 1, 1)
end = datetime(2020, 9, 14)

dis = pdr.DataReader('MSFT', 'yahoo', start=start, end=end)
dis.head()

In [ ]:
# Now apply time series analysis on close. 
data = dis['Close']

plt.figure(figsize=(10,4))
plt.plot(data)
plt.ylabel('End of closing day price', fontsize=16)
plt.title('Stock', fontsize=20)

In [ ]:
#Perform Dickey-Fuller test:
dftest = adfuller(data, autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])

for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
# Apply log to make it linear trend.
stock_ts_log = np.log(data)
plt.plot(stock_ts_log)

In [ ]:
# Normalization
stock_mean = np.mean(stock_ts_log)
stock_std = np.std(stock_ts_log)

stock_ts_norm = (stock_ts_log - stock_mean)/stock_std
plt.plot(stock_ts_norm)

In [ ]:
# Remove trend and seasonality in timeseries data. 
decomposition = seasonal_decompose(stock_ts_norm, model='additive', period = 7)
trend = decomposition.trend.dropna()
seasonal = decomposition.seasonal.dropna()
residual = decomposition.resid.dropna()
plt.figure(figsize=(10,6))

plt.subplot(411)
plt.plot(ts_log, label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
moving_avg = residual.rolling(window=7, center=False).mean()
pyplot.plot(residual)
plt.plot(moving_avg, color='red', label='mean')

In [ ]:
dftest = adfuller(residual, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
# Prepare Train and Test Data
# In timeseries data, make sure when we split the data, order is not changed.
# We need to maintain order of the timeseries.
train_end_date = datetime(2020, 9, 6)
test_date = datetime(2020, 9, 11)

train_data = residual[:train_end_date]
test_data = data[train_end_date + timedelta(days=1):test_date]

train_data.index = pd.DatetimeIndex(train_data.index.values)

print(train_data)
test_data

In [ ]:
# Find the order for AR model using PACF. 
plot_pacf(train_data)
plt.show()

In [ ]:
plot_acf(train_data)
plt.show()

In [ ]:
armodel = ARIMA(train_data, order=(5,0,0))
armodel_fit = armodel.fit()
print(armodel_fit.summary())

In [ ]:
# Forecast for next 12 months
results = armodel_fit.forecast(4)
results

In [ ]:
# Look at neural networks for more advanced approaches. 